In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import time 
from torch import utils
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence

import torchtext
from tqdm import tqdm
from torchtext.datasets import IMDB

from torchtext.datasets.imdb import NUM_LINES
from torchtext.data import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator
from torchtext.data.functional import to_map_style_dataset

import os
import sys
import logging
import logging
logging.basicConfig(
    level=logging.WARN, stream=sys.stdout, \
    format="%(asctime)s (%(module)s:%(lineno)d) %(levelname)s: %(message)s")

VOCAB_SIZE = 15000
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
logging.warning(f"device is {device}")
VOCAB_SIZE = 15000 
BATCH_SIZE = 64

def yeild_tokens(train_data_iter, tokenizer):
    for i, sample in enumerate(train_data_iter):
        label, comment = sample
        yield tokenizer(comment)
        
class TextClassificationModel(nn.Module):
    def __init__(self, vocab_size=VOCAB_SIZE, embed_dim=64, num_class=2):
        super(TextClassificationModel, self).__init__()
        self.embedding = nn.EmbeddingBag(vocab_size, embed_dim, sparse=False).to(device)
        self.fc = nn.Linear(embed_dim, num_class).to(device)
        
    def forward(self, token_index):
        embedded = self.embedding(token_index)  
        return self.fc(embedded)

2023-05-06 07:19:15,806 (2659311876:27) WARNING: device is cuda


In [2]:
train_data_iter = IMDB(root="data", split="train") 
tokenizer = get_tokenizer("basic_english")

vocab = build_vocab_from_iterator(yeild_tokens(train_data_iter, tokenizer), min_freq=20, specials=["<unk>"])
vocab.set_default_index(0)  
print(f'Vocal size: {len(vocab)}')

Vocal size: 13351


In [3]:
def collate_fn(batch):
    target = []
    token_index = []
    max_length = 0  
    for i, (label, comment) in enumerate(batch):
        tokens = tokenizer(comment)
        token_index.append(vocab(tokens)) 
        
        if len(tokens) > max_length:
            max_length = len(tokens)
        
        if label == "pos":
            target.append(0)
        else:
            target.append(1)

    token_index = [index + [0]*(max_length-len(index)) for index in token_index]
    return (torch.tensor(target).to(torch.int64), torch.tensor(token_index).to(torch.int32))

In [4]:
def train(train_data_loader, eval_data_loader, model, optimizer, num_epoch, log_step_interval, save_step_interval, \
          eval_step_interval, save_path, resume=""):
    start_epoch = 0
    start_step = 0
    if resume != "":
        logging.warning(f"loading from {resume}")
        checkpoint = torch.load(resume)
        model.load_state_dict(checkpoint['model_state_dict'])
        optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
        start_epoch = checkpoint['epoch']
        start_step = checkpoint['step']
        
    for epoch_index in tqdm(range(start_epoch, num_epoch), desc="epoch"):
        ema_loss = 0
        num_batches = len(train_data_loader)
        start_time = time.time()
        
        for batch_index, (target, token_index) in enumerate(train_data_loader):
            target = target.to(device)
            token_index = token_index.to(device)
            optimizer.zero_grad()
            step = num_batches*(epoch_index) + batch_index + 1
            logits = model(token_index)

            bce_loss = F.binary_cross_entropy(torch.sigmoid(logits), F.one_hot(target, num_classes=2).to(torch.float32))
            ema_loss = 0.9 * ema_loss + 0.1 * bce_loss  
            bce_loss.backward()
            nn.utils.clip_grad_norm_(model.parameters(), 0.1)  
            optimizer.step()   
            
            if step % log_step_interval == 0:
                logging.warning(f"epoch_index: {epoch_index}, batch_index: {batch_index}, ema_loss: {ema_loss}")
                
            if step % save_step_interval == 0:
                os.makedirs(save_path, exist_ok=True)
                save_file = os.path.join(save_path, f"step_{step}.pt")
                torch.save({
                    "epoch": epoch_index,
                    "step": step,
                    "model_state_dict": model.state_dict(),
                    'optimizer_state_dict': optimizer.state_dict(),
                    'loss': bce_loss,
                }, save_file)
                logging.warning(f"checkpoint has been saved in {save_file}")
            
            if step % eval_step_interval == 0:
                logging.warning("start to do evaluation...")
                model.eval()
                ema_eval_loss = 0
                total_acc_account = 0
                total_account = 0
                for eval_batch_index, (eval_target, eval_token_index) in enumerate(eval_data_loader):
                    eval_target = eval_target.to(device)
                    eval_token_index = eval_token_index.to(device)
                    total_account += eval_target.shape[0]
                    eval_logits = model(eval_token_index)
                    total_acc_account += (torch.argmax(eval_logits, dim=-1) == eval_target).sum().item()
                    eval_bce_loss = F.binary_cross_entropy(torch.sigmoid(eval_logits), F.one_hot(eval_target, num_classes=2).to(torch.float32))
                    ema_eval_loss = 0.9 * ema_eval_loss + 0.1 * eval_bce_loss
                logging.warning(f"ema_eval_loss: {ema_eval_loss}, eval_acc: {total_acc_account / total_account}")
                model.train()
        time_period = time.time()-start_time
        logging.warning(f"time cost is: {time_period}")

In [5]:
model = TextClassificationModel().to(device)
print("Model total parameters:", sum(p.numel() for p in model.parameters()))
resume = ""

Model total parameters: 960130


In [6]:
train_data_iter = IMDB(root="data", split="train") 
train_data_loader = torch.utils.data.DataLoader(
    to_map_style_dataset(train_data_iter), batch_size=BATCH_SIZE, collate_fn=collate_fn, shuffle=True)

eval_data_iter = IMDB(root="data", split="test")  
eval_data_loader = utils.data.DataLoader(
    to_map_style_dataset(eval_data_iter), batch_size=8, collate_fn=collate_fn)

In [7]:
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
train(train_data_loader, eval_data_loader, model, optimizer, num_epoch=10, log_step_interval=20, \
      save_step_interval=500, eval_step_interval=300, save_path="./embed_adm", resume=resume)

epoch:   0%|          | 0/10 [00:00<?, ?it/s]

2023-05-06 07:19:48,481 (2306368053:32) WARNING: epoch_index: 0, batch_index: 19, ema_loss: 0.6318719983100891
2023-05-06 07:19:48,755 (2306368053:32) WARNING: epoch_index: 0, batch_index: 39, ema_loss: 0.7013704180717468
2023-05-06 07:19:49,035 (2306368053:32) WARNING: epoch_index: 0, batch_index: 59, ema_loss: 0.704185426235199
2023-05-06 07:19:49,309 (2306368053:32) WARNING: epoch_index: 0, batch_index: 79, ema_loss: 0.7002986669540405
2023-05-06 07:19:49,580 (2306368053:32) WARNING: epoch_index: 0, batch_index: 99, ema_loss: 0.6969286799430847
2023-05-06 07:19:49,848 (2306368053:32) WARNING: epoch_index: 0, batch_index: 119, ema_loss: 0.6947053074836731
2023-05-06 07:19:50,130 (2306368053:32) WARNING: epoch_index: 0, batch_index: 139, ema_loss: 0.6939979195594788
2023-05-06 07:19:50,412 (2306368053:32) WARNING: epoch_index: 0, batch_index: 159, ema_loss: 0.6935691833496094
2023-05-06 07:19:50,692 (2306368053:32) WARNING: epoch_index: 0, batch_index: 179, ema_loss: 0.693481802940368

epoch:  10%|█         | 1/10 [00:10<01:38, 10.91s/it]

2023-05-06 07:19:59,235 (2306368053:32) WARNING: epoch_index: 1, batch_index: 8, ema_loss: 0.4241696298122406
2023-05-06 07:19:59,519 (2306368053:32) WARNING: epoch_index: 1, batch_index: 28, ema_loss: 0.6600161194801331
2023-05-06 07:19:59,793 (2306368053:32) WARNING: epoch_index: 1, batch_index: 48, ema_loss: 0.6893443465232849
2023-05-06 07:20:00,070 (2306368053:32) WARNING: epoch_index: 1, batch_index: 68, ema_loss: 0.6926725506782532
2023-05-06 07:20:00,349 (2306368053:32) WARNING: epoch_index: 1, batch_index: 88, ema_loss: 0.6929142475128174
2023-05-06 07:20:00,616 (2306368053:32) WARNING: epoch_index: 1, batch_index: 108, ema_loss: 0.691936194896698
2023-05-06 07:20:00,633 (2306368053:44) WARNING: checkpoint has been saved in ./embed_adm/step_500.pt
2023-05-06 07:20:00,909 (2306368053:32) WARNING: epoch_index: 1, batch_index: 128, ema_loss: 0.692777156829834
2023-05-06 07:20:01,179 (2306368053:32) WARNING: epoch_index: 1, batch_index: 148, ema_loss: 0.6924036145210266
2023-05-06

epoch:  20%|██        | 2/10 [00:21<01:27, 10.98s/it]

...

epoch:  80%|████████  | 8/10 [01:41<00:26, 13.05s/it]

2023-05-06 07:21:29,494 (2306368053:32) WARNING: epoch_index: 8, batch_index: 11, ema_loss: 0.49233555793762207
2023-05-06 07:21:29,768 (2306368053:32) WARNING: epoch_index: 8, batch_index: 31, ema_loss: 0.6626986265182495
2023-05-06 07:21:30,038 (2306368053:32) WARNING: epoch_index: 8, batch_index: 51, ema_loss: 0.6827876567840576
2023-05-06 07:21:30,310 (2306368053:32) WARNING: epoch_index: 8, batch_index: 71, ema_loss: 0.6854875683784485
2023-05-06 07:21:30,588 (2306368053:32) WARNING: epoch_index: 8, batch_index: 91, ema_loss: 0.6867401599884033
2023-05-06 07:21:30,865 (2306368053:32) WARNING: epoch_index: 8, batch_index: 111, ema_loss: 0.6865541934967041
2023-05-06 07:21:31,145 (2306368053:32) WARNING: epoch_index: 8, batch_index: 131, ema_loss: 0.6861276030540466
2023-05-06 07:21:31,428 (2306368053:32) WARNING: epoch_index: 8, batch_index: 151, ema_loss: 0.6849938035011292
2023-05-06 07:21:31,704 (2306368053:32) WARNING: epoch_index: 8, batch_index: 171, ema_loss: 0.6857311725616

epoch:  90%|█████████ | 9/10 [01:52<00:12, 12.44s/it]

2023-05-06 07:21:40,429 (2306368053:32) WARNING: epoch_index: 9, batch_index: 0, ema_loss: 0.06869905441999435
2023-05-06 07:21:40,706 (2306368053:32) WARNING: epoch_index: 9, batch_index: 20, ema_loss: 0.6105771660804749
2023-05-06 07:21:40,973 (2306368053:32) WARNING: epoch_index: 9, batch_index: 40, ema_loss: 0.6754571199417114
2023-05-06 07:21:41,241 (2306368053:32) WARNING: epoch_index: 9, batch_index: 60, ema_loss: 0.6822473406791687
2023-05-06 07:21:41,525 (2306368053:32) WARNING: epoch_index: 9, batch_index: 80, ema_loss: 0.6852259635925293
2023-05-06 07:21:41,526 (2306368053:47) WARNING: start to do evaluation...
2023-05-06 07:21:47,292 (2306368053:60) WARNING: ema_eval_loss: 0.6690146327018738, eval_acc: 0.66728
2023-05-06 07:21:47,573 (2306368053:32) WARNING: epoch_index: 9, batch_index: 100, ema_loss: 0.6840384006500244
2023-05-06 07:21:47,841 (2306368053:32) WARNING: epoch_index: 9, batch_index: 120, ema_loss: 0.684909999370575
2023-05-06 07:21:48,110 (2306368053:32) WARNI

epoch: 100%|██████████| 10/10 [02:09<00:00, 12.91s/it]


In [8]:
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4)
train(train_data_loader, eval_data_loader, model, optimizer, num_epoch=10, log_step_interval=20, \
      save_step_interval=500, eval_step_interval=300, save_path="./embed_admw", resume=resume)

epoch:   0%|          | 0/10 [00:00<?, ?it/s]

2023-05-06 07:21:57,624 (2306368053:32) WARNING: epoch_index: 0, batch_index: 19, ema_loss: 0.5998809933662415
2023-05-06 07:21:57,903 (2306368053:32) WARNING: epoch_index: 0, batch_index: 39, ema_loss: 0.6729063391685486
2023-05-06 07:21:58,178 (2306368053:32) WARNING: epoch_index: 0, batch_index: 59, ema_loss: 0.6817951798439026
2023-05-06 07:21:58,449 (2306368053:32) WARNING: epoch_index: 0, batch_index: 79, ema_loss: 0.682714581489563
2023-05-06 07:21:58,722 (2306368053:32) WARNING: epoch_index: 0, batch_index: 99, ema_loss: 0.6839251518249512
2023-05-06 07:21:58,993 (2306368053:32) WARNING: epoch_index: 0, batch_index: 119, ema_loss: 0.6837183833122253
2023-05-06 07:21:59,275 (2306368053:32) WARNING: epoch_index: 0, batch_index: 139, ema_loss: 0.6833877563476562
2023-05-06 07:21:59,554 (2306368053:32) WARNING: epoch_index: 0, batch_index: 159, ema_loss: 0.6833356618881226
2023-05-06 07:21:59,825 (2306368053:32) WARNING: epoch_index: 0, batch_index: 179, ema_loss: 0.681990265846252

epoch:  10%|█         | 1/10 [00:10<01:38, 10.92s/it]

2023-05-06 07:22:08,399 (2306368053:32) WARNING: epoch_index: 1, batch_index: 8, ema_loss: 0.418472558259964
2023-05-06 07:22:08,675 (2306368053:32) WARNING: epoch_index: 1, batch_index: 28, ema_loss: 0.6493127942085266
2023-05-06 07:22:08,944 (2306368053:32) WARNING: epoch_index: 1, batch_index: 48, ema_loss: 0.6756512522697449
2023-05-06 07:22:09,207 (2306368053:32) WARNING: epoch_index: 1, batch_index: 68, ema_loss: 0.6810343265533447
2023-05-06 07:22:09,487 (2306368053:32) WARNING: epoch_index: 1, batch_index: 88, ema_loss: 0.6827625036239624
2023-05-06 07:22:09,764 (2306368053:32) WARNING: epoch_index: 1, batch_index: 108, ema_loss: 0.6823949813842773
2023-05-06 07:22:09,781 (2306368053:44) WARNING: checkpoint has been saved in ./embed_admw/step_500.pt
2023-05-06 07:22:10,070 (2306368053:32) WARNING: epoch_index: 1, batch_index: 128, ema_loss: 0.6833586692810059
2023-05-06 07:22:10,355 (2306368053:32) WARNING: epoch_index: 1, batch_index: 148, ema_loss: 0.6815834045410156
2023-05-

epoch:  20%|██        | 2/10 [00:21<01:27, 10.99s/it]

...

epoch:  80%|████████  | 8/10 [01:39<00:25, 12.72s/it]

2023-05-06 07:23:36,806 (2306368053:32) WARNING: epoch_index: 8, batch_index: 11, ema_loss: 0.47389909625053406
2023-05-06 07:23:37,090 (2306368053:32) WARNING: epoch_index: 8, batch_index: 31, ema_loss: 0.6426214575767517
2023-05-06 07:23:37,373 (2306368053:32) WARNING: epoch_index: 8, batch_index: 51, ema_loss: 0.6644608974456787
2023-05-06 07:23:37,645 (2306368053:32) WARNING: epoch_index: 8, batch_index: 71, ema_loss: 0.6591585278511047
2023-05-06 07:23:37,913 (2306368053:32) WARNING: epoch_index: 8, batch_index: 91, ema_loss: 0.6613760590553284
2023-05-06 07:23:38,189 (2306368053:32) WARNING: epoch_index: 8, batch_index: 111, ema_loss: 0.665524959564209
2023-05-06 07:23:38,465 (2306368053:32) WARNING: epoch_index: 8, batch_index: 131, ema_loss: 0.6631430983543396
2023-05-06 07:23:38,747 (2306368053:32) WARNING: epoch_index: 8, batch_index: 151, ema_loss: 0.6614740490913391
2023-05-06 07:23:39,020 (2306368053:32) WARNING: epoch_index: 8, batch_index: 171, ema_loss: 0.66308033466339

epoch:  90%|█████████ | 9/10 [01:50<00:12, 12.22s/it]

2023-05-06 07:23:47,782 (2306368053:32) WARNING: epoch_index: 9, batch_index: 0, ema_loss: 0.06504123657941818
2023-05-06 07:23:48,060 (2306368053:32) WARNING: epoch_index: 9, batch_index: 20, ema_loss: 0.5884624719619751
2023-05-06 07:23:48,338 (2306368053:32) WARNING: epoch_index: 9, batch_index: 40, ema_loss: 0.6536043882369995
2023-05-06 07:23:48,616 (2306368053:32) WARNING: epoch_index: 9, batch_index: 60, ema_loss: 0.661299467086792
2023-05-06 07:23:48,886 (2306368053:32) WARNING: epoch_index: 9, batch_index: 80, ema_loss: 0.6597021222114563
2023-05-06 07:23:48,886 (2306368053:47) WARNING: start to do evaluation...
2023-05-06 07:23:54,599 (2306368053:60) WARNING: ema_eval_loss: 0.6595385670661926, eval_acc: 0.7156
2023-05-06 07:23:54,873 (2306368053:32) WARNING: epoch_index: 9, batch_index: 100, ema_loss: 0.6590096354484558
2023-05-06 07:23:55,156 (2306368053:32) WARNING: epoch_index: 9, batch_index: 120, ema_loss: 0.6604630351066589
2023-05-06 07:23:55,425 (2306368053:32) WARNIN

epoch: 100%|██████████| 10/10 [02:07<00:00, 12.72s/it]


In [9]:
optimizer = torch.optim.SGD(model.parameters(), lr=1e-4)
train(train_data_loader, eval_data_loader, model, optimizer, num_epoch=10, log_step_interval=20, \
      save_step_interval=500, eval_step_interval=300, save_path="./embed_sgd", resume=resume)

epoch:   0%|          | 0/10 [00:00<?, ?it/s]

2023-05-06 07:24:04,920 (2306368053:32) WARNING: epoch_index: 0, batch_index: 19, ema_loss: 0.5798764228820801
2023-05-06 07:24:05,193 (2306368053:32) WARNING: epoch_index: 0, batch_index: 39, ema_loss: 0.645302414894104
2023-05-06 07:24:05,480 (2306368053:32) WARNING: epoch_index: 0, batch_index: 59, ema_loss: 0.6550522446632385
2023-05-06 07:24:05,756 (2306368053:32) WARNING: epoch_index: 0, batch_index: 79, ema_loss: 0.6586650609970093
2023-05-06 07:24:06,018 (2306368053:32) WARNING: epoch_index: 0, batch_index: 99, ema_loss: 0.6567069888114929
2023-05-06 07:24:06,295 (2306368053:32) WARNING: epoch_index: 0, batch_index: 119, ema_loss: 0.659611165523529
2023-05-06 07:24:06,571 (2306368053:32) WARNING: epoch_index: 0, batch_index: 139, ema_loss: 0.6605455279350281
2023-05-06 07:24:06,836 (2306368053:32) WARNING: epoch_index: 0, batch_index: 159, ema_loss: 0.6583914160728455
2023-05-06 07:24:07,114 (2306368053:32) WARNING: epoch_index: 0, batch_index: 179, ema_loss: 0.6573147177696228

epoch:  10%|█         | 1/10 [00:10<01:37, 10.82s/it]

2023-05-06 07:24:15,582 (2306368053:32) WARNING: epoch_index: 1, batch_index: 8, ema_loss: 0.39887022972106934
2023-05-06 07:24:15,850 (2306368053:32) WARNING: epoch_index: 1, batch_index: 28, ema_loss: 0.6268767714500427
2023-05-06 07:24:16,110 (2306368053:32) WARNING: epoch_index: 1, batch_index: 48, ema_loss: 0.6523779630661011
2023-05-06 07:24:16,374 (2306368053:32) WARNING: epoch_index: 1, batch_index: 68, ema_loss: 0.6580240726470947
2023-05-06 07:24:16,648 (2306368053:32) WARNING: epoch_index: 1, batch_index: 88, ema_loss: 0.6599518060684204
2023-05-06 07:24:16,921 (2306368053:32) WARNING: epoch_index: 1, batch_index: 108, ema_loss: 0.6612637639045715
2023-05-06 07:24:16,929 (2306368053:44) WARNING: checkpoint has been saved in ./embed_sgd/step_500.pt
2023-05-06 07:24:17,213 (2306368053:32) WARNING: epoch_index: 1, batch_index: 128, ema_loss: 0.6606841087341309
2023-05-06 07:24:17,484 (2306368053:32) WARNING: epoch_index: 1, batch_index: 148, ema_loss: 0.657292366027832
2023-05-

epoch:  20%|██        | 2/10 [00:21<01:27, 10.92s/it]

...

epoch:  80%|████████  | 8/10 [01:38<00:25, 12.73s/it]

2023-05-06 07:25:43,704 (2306368053:32) WARNING: epoch_index: 8, batch_index: 11, ema_loss: 0.4730377793312073
2023-05-06 07:25:43,981 (2306368053:32) WARNING: epoch_index: 8, batch_index: 31, ema_loss: 0.633831262588501
2023-05-06 07:25:44,260 (2306368053:32) WARNING: epoch_index: 8, batch_index: 51, ema_loss: 0.653315544128418
2023-05-06 07:25:44,522 (2306368053:32) WARNING: epoch_index: 8, batch_index: 71, ema_loss: 0.6541838049888611
2023-05-06 07:25:44,801 (2306368053:32) WARNING: epoch_index: 8, batch_index: 91, ema_loss: 0.6587887406349182
2023-05-06 07:25:45,091 (2306368053:32) WARNING: epoch_index: 8, batch_index: 111, ema_loss: 0.6581243872642517
2023-05-06 07:25:45,369 (2306368053:32) WARNING: epoch_index: 8, batch_index: 131, ema_loss: 0.6577311754226685
2023-05-06 07:25:45,649 (2306368053:32) WARNING: epoch_index: 8, batch_index: 151, ema_loss: 0.6605755090713501
2023-05-06 07:25:45,919 (2306368053:32) WARNING: epoch_index: 8, batch_index: 171, ema_loss: 0.6597784757614136

epoch:  90%|█████████ | 9/10 [01:49<00:12, 12.19s/it]

2023-05-06 07:25:54,556 (2306368053:32) WARNING: epoch_index: 9, batch_index: 0, ema_loss: 0.06533627957105637
2023-05-06 07:25:54,819 (2306368053:32) WARNING: epoch_index: 9, batch_index: 20, ema_loss: 0.5839166045188904
2023-05-06 07:25:55,083 (2306368053:32) WARNING: epoch_index: 9, batch_index: 40, ema_loss: 0.6476113796234131
2023-05-06 07:25:55,340 (2306368053:32) WARNING: epoch_index: 9, batch_index: 60, ema_loss: 0.653864324092865
2023-05-06 07:25:55,611 (2306368053:32) WARNING: epoch_index: 9, batch_index: 80, ema_loss: 0.6571338772773743
2023-05-06 07:25:55,612 (2306368053:47) WARNING: start to do evaluation...
2023-05-06 07:26:01,338 (2306368053:60) WARNING: ema_eval_loss: 0.6484620571136475, eval_acc: 0.72476
2023-05-06 07:26:01,622 (2306368053:32) WARNING: epoch_index: 9, batch_index: 100, ema_loss: 0.6572405695915222
2023-05-06 07:26:01,903 (2306368053:32) WARNING: epoch_index: 9, batch_index: 120, ema_loss: 0.6594080924987793
2023-05-06 07:26:02,200 (2306368053:32) WARNI

epoch: 100%|██████████| 10/10 [02:06<00:00, 12.66s/it]
